# TrainingServingSkew
* Moniitors whether the alphas generated for inference are identical 
    to the alphas generated during training

In [1]:
import NBInclude: @nbinclude
import Statistics: mean
@nbinclude("Alpha.ipynb");

In [2]:
function read_training_alpha(alpha, split)
    uid = parse(Int, get_recommendee_username()) + 1
    df = read_alpha(alpha, "validation", true)
    mask = df.user .== uid
    df = RatingsDataset(df.user[mask], df.item[mask], df.rating[mask])
    df.user .= 1
    df
end;

In [3]:
function maximum_error(alpha, split)
    serving = read_recommendee_alpha(alpha, "all")
    training = read_training_alpha(alpha, split)
    abs_errors = abs.(training.rating .- serving.rating[training.item])
    rel_errors = abs.(training.rating ./ serving.rating[training.item] .- 1)
    maximum(vcat(abs_errors, rel_errors))
end;

In [4]:
function average_abs_error(alpha, split)
    serving = read_recommendee_alpha(alpha, "all")
    training = read_training_alpha(alpha, split)
    abs_errors = abs.(training.rating .- serving.rating[training.item])
    mean(abs_errors)
end;

In [5]:
function average_rel_error(alpha, split)
    serving = read_recommendee_alpha(alpha, "all")
    training = read_training_alpha(alpha, split)
    abs_errors = abs.(training.rating ./ serving.rating[training.item] .- 1)
    mean(abs_errors)
end;

In [6]:
function display_errors(alphas)
    alpha_padding = maximum(length.(alphas))
    number_padding = 16

    header_1 = rpad("Alpha", alpha_padding)
    header_2 = rpad("Avg Abs Error", number_padding)
    header_3 = rpad("Avg Rel Error", number_padding)
    header_4 = rpad("Max Error", number_padding)
    @info "$header_1 $header_2 $header_3 $header_4"

    for alpha in alphas
        avg_abs_error = rpad(average_abs_error(alpha, "training"), number_padding)
        avg_rel_error = rpad(average_rel_error(alpha, "training"), number_padding)
        max_error = rpad(maximum_error(alpha, "training"), number_padding)
        alpha = rpad(alpha, alpha_padding)
        @info "$alpha $avg_abs_error $avg_rel_error $max_error"
    end
end;

In [7]:
potential_alphas = readdir(recommendee_alpha_basepath())
alphas = [
    x for
    x in potential_alphas if ispath("$(recommendee_alpha_basepath())/$(x)/alpha.jld2")
]
ensemble_alphas = ["ErrorExplicit", "ErrorImplicit", "Explicit", "NonlinearExplicit"]
inference_only_alphas = ["ItemCFRelated", "BPR"]
standard_alphas = [x for x in alphas if x ∉ ensemble_alphas && x ∉ inference_only_alphas];

In [8]:
display_errors(standard_alphas)

[ Info: 20220630 10:49:21 Alpha                             Avg Abs Error    Avg Rel Error    Max Error       
[ Info: 20220630 10:49:24 ExplicitItemCF                    1.0281801e-6     1.3709068e-6     1.7881393e-6    
[ Info: 20220630 10:49:25 ExplicitUserItemBiases            1.9073486e-6     2.3841858e-7     1.9073486e-6    
[ Info: 20220630 10:49:25 LinearExplicit                    0.00871706       0.0010354221     0.010548592     
[ Info: 20220630 10:49:25 LinearImplicit                    0.0011766818     0.09194863       0.15807748      
[ Info: 20220630 10:49:26 NeuralExplicitMatrixFactorization 0.0332275        0.22907925       0.40753388      
[ Info: 20220630 10:49:26 NeuralImplicitMatrixFactorization 0.0012201887     0.093682885      0.15813112      
[ Info: 20220630 10:49:26 NeuralImplicitUserItemBiases      1.2369128e-10    1.4901161e-7     1.7881393e-7    
[ Info: 20220630 10:49:27 SeenItemsCount                    0.0              0.0              0.0             


In [9]:
display_errors(ensemble_alphas)

[ Info: 20220630 10:49:27 Alpha             Avg Abs Error    Avg Rel Error    Max Error       
[ Info: 20220630 10:49:27 ErrorExplicit     0.014685661      0.025528729      0.050234795     
[ Info: 20220630 10:49:27 ErrorImplicit     0.00086967647    0.0028087795     0.005617559     
[ Info: 20220630 10:49:28 Explicit          0.010331392      0.0011695027     0.019058228     
[ Info: 20220630 10:49:28 NonlinearExplicit 0.0190483        0.28568178       0.5386658       
